## Evaluate

$ 5x^{3} - 4x^{2}y^{2} + 13xy^{2} + x^{2} -10y = out $

$ v_1 = xx $

$ v_2 = yy $

$ v_3 = 5xv_1 $

$ v_4 = 4v_1v_2 $

$ out -v_3 + v_4 - v_1 + 10y = 13xv_2 $

$ w = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4
\end{bmatrix} $



$ L = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 5 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 4 & 0 & 0 & 0 \\
0 & 0 & 13 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix} $


$ R = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
\end{bmatrix} $


$ O = \begin{bmatrix}
1 & out & x & y & v_1 & v_2 & v_3 & v_4 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 10 & -1 & 0 & -1 & 1 \\
\end{bmatrix} $


## R1CS

In [6]:
import galois
import numpy as np

p = 71
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(p)

x = FP(2)
y = FP(3)

v1 = x * x
v2 = y * y
v3 = 5 * x * v1
v4 = 4 * v1 * v2
out = 5*x**3 - 4*x**2*y**2 + 13*x*y**2 + x**2 - 10*y

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w =", w)

R = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 5, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 4, 0, 0, 0],
         [0, 0, 13, 0, 0, 0, 0, 0]])

L = FP([[0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0]])

O = FP([[0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 1],
         [0, 1, 0, 10, FP(p - 1), 0, FP(p - 1), 1]])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)


w = [  1 104   2   3   4   9  40 144]
Lw = [2 3 4 9 9]
Rw = [ 2  3 10 16 26]
Lw * Rw = [  4   9  40 144 234]
Ow =      [  4   9  40 144 234]


## QAP

In [7]:
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''L
{Lp}
''')

print(f'''R
{Rp}
''')

print(f'''O
{Op}
''')

L
[[                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            5
   9120101196599698009269335727190531286895151833506680976540918411073253539834
    91201011965996980092693357271905

In [8]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
tau = FP(20)

print(f"τ = {tau}")

T = galois.Poly([1, p-1], field=FP)
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly([1, p-i], field=FP)

print("\nT = ", T)
for i in range(1, L.shape[0] + 2):
    print(f"T({i}) = ", T(i))
    if i == L.shape[0]:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

Setup phase
----------
Toxic waste:
τ = 20

T =  x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 21888242871839275222246405745257275088548364400416034343698204186575808495497
T(1) =  0
T(2) =  0
T(3) =  0
T(4) =  0
T(5) =  0
----------
T(6) =  120

T(τ) = 1395360


In [9]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[T(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[τ]G2 = {[normalize(point) for point in tau_G2]}")

Trusted setup:
----------
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)]
[T(τ)]G1 = [(1641247283492879903468444805169804215277964208279225379119694118848884481979, 18733245328562972535068072505811612803902036184072974454572123739995203670419), (1282524939337382483469274134339301655621870987447481453798307725808867601190, 155391820121621028156365747107525575876406255471822024656389149920860

In [10]:
U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)

H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

assert rem == 0

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v - _w == ht, f"{u} * {v} - {_w} != {ht}"

U =  11856131555579607412050136445347690672963697383558685269503193934395229601792x^4 + 20064222632519335620392538599819168831169334033714698148390020504361157787655x^3 + 20976232752179305421319472172538221959858849217065366246044112345468483141610x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955812x + 21888242871839275222246405745257275088548364400416034343698204186575808495601
V =  10944121435919637611123202872628637544274182200208017171849102093287904247809x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415930x^3 + 10944121435919637611123202872628637544274182200208017171849102093287904247836x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079635x + 26
W =  12768141675239577212977070018066743801653212566909353367157285775502554955771x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602831936x^3 + 9120101196599698009269335727190531286895151833506680976540918411073253539611x^2 

In [11]:
def evaluate_poly(poly, trusted_points, verbose=False):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"

    if verbose:
        [print(normalize(point)) for point in trusted_points]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    if verbose:
        print("-"*10)
        print(normalize(evaluation))
    return evaluation

print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
B_G1 = evaluate_poly(V, tau_G1)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")


print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Pairing check failed!"
print("Pairing check passed!")


Proof generation:
----------
[A]G1 = (19092006581455788758709004813424108450475230671546198110182704126760952021248, 18428185916649502171614192229986655674799279684527591370328182794110727996633)
[B]G2 = ((1110332524507442648511549408896049077062269578877062826069065960274388112308, 15815785354885964222010325771656100864105333417560377595802485750386873282739), (20784382045877636010618629654573620888044404319093695781168988411617616204166, 5234804291052944426941184034424257962428641145809086397589880058685491457835))
[C]G1 = (21755526246297599392782387322262927251662305599666002632514868138515690603377, 19883332083442129478217826420060112230198011363938980948134718366700920887106)

Proof verification:
----------
Pairing check passed!


## Test: shifting prover's work to verifier
pairing(A, B) = pairing(α, β) + pairing(β, A) + pairing(α, B) + pairing(C, G2)

In [7]:
alpha = FP(17)
beta = FP(117)

alpha_G1 = multiply(G1, int(alpha))
beta_G2 = multiply(G2, int(beta))

A_G1_aplha = add(A_G1, alpha_G1)
B_G2_beta = add(B_G2, beta_G2)

lhs = pairing(B_G2_beta, A_G1_aplha)
rhs1 = pairing(beta_G2, alpha_G1)
rhs2 = pairing(cG1, )

# with open("Verifier2.sol.template", "r") as f:
#     from string import Template
#     template = Template(f.read())
#     variables = {
#         "aG1_x": normalize(A_G1)[0],
#         "aG1_y": normalize(A_G1)[1],
#         "aAlphaG1_x": normalize(A_G1_aplha)[0],
#         "aAlphaG1_y": normalize(A_G1_aplha)[1],
#         "bG2_x1": normalize(B_G2)[0].coeffs[0],
#         "bG2_x2": normalize(B_G2)[0].coeffs[1],
#         "bG2_y1": normalize(B_G2)[1].coeffs[0],
#         "bG2_y2": normalize(B_G2)[1].coeffs[1],
#         "bBetaG2_x1": normalize(B_G2_beta)[0].coeffs[0],
#         "bBetaG2_x2": normalize(B_G2_beta)[0].coeffs[1],
#         "bBetaG2_y1": normalize(B_G2_beta)[1].coeffs[0],
#         "bBetaG2_y2": normalize(B_G2_beta)[1].coeffs[1],
#         "cG1_x": normalize(C_G1)[0],
#         "cG1_y": normalize(C_G1)[1],
#         "alphaG1_x": normalize(alpha_G1)[0],
#         "alphaG1_y": normalize(alpha_G1)[1],
#         "betaG2_x1": normalize(beta_G2)[0].coeffs[0],
#         "betaG2_x2": normalize(beta_G2)[0].coeffs[1],
#         "betaG2_y1": normalize(beta_G2)[1].coeffs[0],
#         "betaG2_y2": normalize(beta_G2)[1].coeffs[1],
#     }
#     output = template.substitute(variables)

# with open("Verifier2.sol", "w") as f:
#     f.write(output)

## α, β

In [8]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"τ = {tau}")

Setup phase
----------
Toxic waste:
α = 2
β = 3
τ = 20


In [9]:
def evaluate_poly_list(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append( galois.Poly(mtx[i][::-1]) )
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

beta_L = beta * Lp
alpha_R = alpha * Rp

print("βL\n", beta_L)
print("αR\n", alpha_R)
print("O\n", Op)

K = beta_L + alpha_R + Op 
print("K = βL + αR + O\n", K)

Kp = to_poly(K)
print_poly("K", Kp)

print("K evaluations:")
K_eval = evaluate_poly_list(Kp, tau)
print([int(k) for k in K_eval])

βL
 [[                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                           15
   5472060717959818805561601436314318772137091100104008585924551046643952123885
   2736030358979909402780800718157

In [10]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize, eq

# G1[α]
alpha_G1 = multiply(G1, int(alpha))
# G1[β]
beta_G1 = multiply(G1, int(beta))
# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[βU0(τ) + αV0(τ) + W0(τ)], G1[βU1(τ) + αV1(τ) + W1(τ)], ..., G1[βUd(τ) + αVd(τ) + Wd(τ)]
k_G1 = [multiply(G1, int(k)) for k in K_eval]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[α]
beta_G2 = multiply(G2, int(beta))
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G1)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[k]G1 = {[normalize(point) for point in k_G1]}")
print(f"[τT(τ)]G1 = {[normalize(point) for point in target_G1]}")

print(f"\n[β]G2 = {normalize(beta_G2)}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")

Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480515441416830039777911637913418824951667761761)
[τ]G1 = [(1, 2), (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992), (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714), (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677), (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843

In [11]:
print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G1[A] = G1[A] + G1[α]
A_G1 = add(A_G1, alpha_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G2[B] = G2[B] + G2[β]
B_G2 = add(B_G2, beta_G2)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)
# w0 * G1[k0] + w1 * G1[k1] + ... + wd-1 * G1[kd-1]
assert len(w) == len(k_G1), "Polynomial degree mismatch!"
K_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_G1, w)]
K_G1 = K_G1_terms[0]
for i in range(1, len(K_G1_terms)):
    K_G1 = add(K_G1, K_G1_terms[i])

C_G1 = add(HT_G1, K_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")
print("-" * 10)
print("Verifier computes:")
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G1)}")


Proof generation:
----------
[A]G1 = (1386024683863798223806715768494499062004541323940181712423964207525793364711, 140843658074195725477996606607942635995385653032596678292076222071924134)
[B]G2 = ((2268131489099612700799057509317325492629365410973939712610009952960246451283, 5460202898618824449851004635208915161845252587893842541197344609059159079041), (21214978237453897862935673366579439999311734514850735211426933843714547507738, 20956792494964230503385709419527449681227575755609196682056154158621712303733))
[C]G1 = (16949762960944359451850183815207514502115470096021213979561775521498090739408, 19195070842803981584459765300451665511335687015857205049191967196163351934749)
----------
Verifier computes:
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 193215337665523688609465524374805

In [12]:
from string import Template

with open("Verifier.sol.template", "r") as f:
    template = Template(f.read())
    variables = {
        "aG1_x": normalize(neg(A_G1))[0],
        "aG1_y": normalize(neg(A_G1))[1],
        "bG2_x1": normalize(B_G2)[0].coeffs[0],
        "bG2_x2": normalize(B_G2)[0].coeffs[1],
        "bG2_y1": normalize(B_G2)[1].coeffs[0],
        "bG2_y2": normalize(B_G2)[1].coeffs[1],
        "cG1_x": normalize(C_G1)[0],
        "cG1_y": normalize(C_G1)[1],
        "alphaG1_x": normalize(alpha_G1)[0],
        "alphaG1_y": normalize(alpha_G1)[1],
        "betaG2_x1": normalize(beta_G2)[0].coeffs[0],
        "betaG2_x2": normalize(beta_G2)[0].coeffs[1],
        "betaG2_y1": normalize(beta_G2)[1].coeffs[0],
        "betaG2_y2": normalize(beta_G2)[1].coeffs[1],
    }
    output = template.substitute(variables)

with open("Verifier.sol", "w") as f:
    f.write(output)

## α, β + public input + witness(=private input)

In [13]:
# DEBUG

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v == _w + ht

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

c = (beta * u2 + alpha * v2 + w2) + ht 
k = (beta * u1 + alpha * v1 + w1)

assert (u + alpha) * (v + beta) == alpha * beta + k + c

In [14]:
k_pub_G1, k_priv_G1 = k_G1[:2], k_G1[2:]
pub_input, priv_input = w[:2], w[2:]

print(f"[k_pub]G1 = {[normalize(point) for point in k_pub_G1]}")
print(f"[k_priv]G1 = {[normalize(point) for point in k_priv_G1]}")

print(f"pub_input = {pub_input}")
print(f"priv_input = {priv_input}")

[k_pub]G1 = [(0, 0), (17518925876389381914155930110408777142106130111113028150026931997812097774535, 8858392472586588859720780881006689049064823023738566495968587324881339689331)]
[k_priv]G1 = [(7776932200273552787128105306335292200128839089213803200158363828327183775433, 1676473267286376034272794912591089409235063853265136908465687242866588965033), (9255362000568086969426581910999695218767488354011423148343055923150063610530, 19663255005511524857343047157430517662352787346134121583487236661223935168532), (4584923233296350927122616402897357895248802996295595163135244549277455848652, 5242307852868269725944901706296941327208644924473751582476401123626838714022), (16055450244558982737197017116370443036836369038269239292860030068216908828776, 16459830096784505310827542419419419268816240069809368541804931193116745432653), (9207874645969801767372932125425645087469609714108091562596279219993142537032, 21595782772050531164058871317405782630067754355442908985935623343510342105362), (17872882827

In [15]:
K_pub_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_pub_G1, pub_input)]
K_pub_G1 = K_pub_G1_terms[0]
for i in range(1, len(K_pub_G1_terms)):
    K_pub_G1 = add(K_pub_G1, K_pub_G1_terms[i])

print(f"[K_pub]G1 = {normalize(K_pub_G1)}")

K_priv_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_priv_G1, priv_input)]
K_priv_G1 = K_priv_G1_terms[0]
for i in range(1, len(K_priv_G1_terms)):
    K_priv_G1 = add(K_priv_G1, K_priv_G1_terms[i])

print(f"[K_priv]G1 = {normalize(K_priv_G1)}")

[K_pub]G1 = (2285801182806610770657419494535950855770340693072843615122889807967969315919, 18123511326312116315687556134605188595130818916602058636606420458000478513163)
[K_priv]G1 = (13036706852709674721939504211579180811887492113170902152056464186779178688019, 8353218878669375661419844440357764499286581534205369894320348990558116407844)


In [16]:
C_G1 = add(HT_G1, K_priv_G1)

print("\nProof generation:")
print("-"*10)
print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")
print("-" * 10)
print("Verifier uses:")
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G1 = {normalize(beta_G1)}")
print(f"[K terms]G1 = {[normalize(p) for p in k_pub_G1]}")


Proof generation:
----------
[A]G1 = (1386024683863798223806715768494499062004541323940181712423964207525793364711, 140843658074195725477996606607942635995385653032596678292076222071924134)
[B]G2 = ((2268131489099612700799057509317325492629365410973939712610009952960246451283, 5460202898618824449851004635208915161845252587893842541197344609059159079041), (21214978237453897862935673366579439999311734514850735211426933843714547507738, 20956792494964230503385709419527449681227575755609196682056154158621712303733))
[C]G1 = (3007959184562619447308758040351540166633693812660366965459472595743782653474, 21613286137401577200129067304160980059729655631955179340440670659782625352184)
----------
Verifier uses:
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480515441

In [17]:
from string import Template

k1 = normalize(k_pub_G1[0])
k2 = normalize(k_pub_G1[1])

with open("VerifierPublicInput.sol.template", "r") as f:
    template = Template(f.read())
    variables = {
        "aG1_x": normalize(A_G1)[0],
        "aG1_y": normalize(A_G1)[1],
        "bG2_x1": normalize(B_G2)[0].coeffs[0],
        "bG2_x2": normalize(B_G2)[0].coeffs[1],
        "bG2_y1": normalize(B_G2)[1].coeffs[0],
        "bG2_y2": normalize(B_G2)[1].coeffs[1],
        "cG1_x": normalize(C_G1)[0],
        "cG1_y": normalize(C_G1)[1],
        "alphaG1_x": normalize(alpha_G1)[0],
        "alphaG1_y": normalize(alpha_G1)[1],
        "betaG2_x1": normalize(beta_G2)[0].coeffs[0],
        "betaG2_x2": normalize(beta_G2)[0].coeffs[1],
        "betaG2_y1": normalize(beta_G2)[1].coeffs[0],
        "betaG2_y2": normalize(beta_G2)[1].coeffs[1],
        "k1G1_x": k1[0],
        "k1G1_y": k1[1],
        "k2G1_x": k2[0],
        "k2G1_y": k2[1],
        "one": pub_input[0],
        "out": pub_input[1],
    }
    output = template.substitute(variables)

with open("VerifierPublicInput.sol", "w") as f:
    f.write(output)

## gamma, delta (γ, δ)

In [18]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20


In [19]:
def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

# U, V, W - witness already computed

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v == _w + ht

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

c = (beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

a = u + alpha
b = v + beta

assert a * b == alpha * beta + k * gamma + c * delta

alpha_G1 = multiply(G1, int(alpha))
beta_G2 = multiply(G2, int(beta))
gamma_G2 = multiply(G2, int(gamma))
delta_G2 = multiply(G2, int(delta))

In [20]:
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

powers_tauTtau_div_delta = [(tau**i * T_tau) / delta for i in range(0, T.degree - 1)]
target_G1 = [multiply(G1, int(pTd)) for pTd in powers_tauTtau_div_delta]

assert len(target_G1) == len(H.coefficients()), f"target_G1 length mismatch! {len(target_G1)} != {len(H.coefficients())}"

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {normalize(alpha_G1)}")
print(f"[β]G2 = {normalize(beta_G2)}")
print(f"[γ]G2 = {normalize(gamma_G2)}")
print(f"[δ]G2 = {normalize(delta_G2)}")
print(f"[τ]G1 = {[normalize(point) for point in tau_G1]}")
print(f"[τ]G2 = {[normalize(point) for point in tau_G2]}")
print(f"[τT(τ)/δ]G1 = {[normalize(point) for point in target_G1]}")

Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G2 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
[γ]G2 = ((18936818173480011669507163011118288089468827259971823710084038754632518263340, 18556147586753789634670778212244811446448229326945855846642767021074501673839), (18825831177813899069786213865729385895767511805925522466244528695074736584695, 13775476761357503446238925910346030822904460488609979964814810757616608848118))
[δ]G2 = ((20954117799226682825035885491234530437475518021362091509513177301640194298072, 4540444681147253467785307942530223364530218361853237193970751657229

In [21]:
w_pub = w[:2]
w_priv = w[2:]

K_gamma, K_delta = [k/gamma for k in K_eval[:2]], [k/delta for k in K_eval[2:]]

print(f"K/γ = {[int(k) for k in K_gamma]}")
print(f"K/δ = {[int(k) for k in K_delta]}")

K_gamma_G1 = [multiply(G1, int(k)) for k in K_gamma]
K_delta_G1 = [multiply(G1, int(k)) for k in K_delta]

print(f"[K/γ]G1 = {[normalize(point) for point in K_gamma_G1]}")
print(f"[K/δ]G1 = {[normalize(point) for point in K_delta_G1]}")

# [K/γ*w]G1
Kw_gamma_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_gamma_G1, w_pub)]
Kw_gamma_G1 = Kw_gamma_G1_terms[0]
for i in range(1, len(Kw_gamma_G1_terms)):
    Kw_gamma_G1 = add(Kw_gamma_G1, Kw_gamma_G1_terms[i])

print(f"[K/γ*w]G1 = {normalize(Kw_gamma_G1)}")

# [K/δ*w]G1
Kw_delta_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_delta_G1, w_priv)]
Kw_delta_G1 = Kw_delta_G1_terms[0]
for i in range(1, len(Kw_delta_G1_terms)):
    Kw_delta_G1 = add(Kw_delta_G1, Kw_delta_G1_terms[i])

print(f"[K/δ*w]G1 = {normalize(Kw_delta_G1)}")

K/γ = [0, 969]
K/δ = [8755297148735710088898562298102910035419345760166413737479281674630323462502, 21888242871839275222246405745257275088548364400416034343698204186575808490449, 13132945723103565133347843447154365053129018640249620606218922511945485086263, 4377648574367855044449281149051455017709672880083206868739640837315161690144, 13132945723103565133347843447154365053129018640249620606218922511945485100699, 8755297148735710088898562298102910035419345760166413737479281674630323396115]
[K/γ]G1 = [(0, 0), (1540147615742092855122362390273215696202300989980535937807544369063691019287, 916302540883947080468652904864832717784863855769927076757632797189693955321)]
[K/δ]G1 = [(16669446524815121333219267530318864581047933015823499703865046203789737572536, 5157664181045350924302328617433280138295240518266317101128319524740459869530), (11685112056990319855844906381401928404871221435467072756674565751991593098502, 1024702697817543489078129137147119462713200516932679197626440621457145655412), (4

In [22]:
HT_G1 = evaluate_poly(H, target_G1)
print(f"\n[τT(τ)/δ]G1 = {normalize(HT_G1)}")

assert pairing(G2, multiply(G1, int(ht/delta))) == pairing(G2, HT_G1)


[τT(τ)/δ]G1 = (7777808773035223595297985042785015077461049335879668711794115277443137103947, 8782695007337652766659561293415156424199863414513788180389485202917517265161)


In [23]:
C_G1 = add(Kw_delta_G1, HT_G1)
K_G1 = Kw_gamma_G1

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)

k1 = normalize(K_gamma_G1[0])
k2 = normalize(K_gamma_G1[1])

with open("VerifierPublicInputGammaDelta.sol.template", "r") as f:
    template = Template(f.read())
    variables = {
        "aG1_x": normalize(A_G1)[0],
        "aG1_y": normalize(A_G1)[1],
        "bG2_x1": normalize(B_G2)[0].coeffs[0],
        "bG2_x2": normalize(B_G2)[0].coeffs[1],
        "bG2_y1": normalize(B_G2)[1].coeffs[0],
        "bG2_y2": normalize(B_G2)[1].coeffs[1],
        "cG1_x": normalize(C_G1)[0],
        "cG1_y": normalize(C_G1)[1],
        "alphaG1_x": normalize(alpha_G1)[0],
        "alphaG1_y": normalize(alpha_G1)[1],
        "betaG2_x1": normalize(beta_G2)[0].coeffs[0],
        "betaG2_x2": normalize(beta_G2)[0].coeffs[1],
        "betaG2_y1": normalize(beta_G2)[1].coeffs[0],
        "betaG2_y2": normalize(beta_G2)[1].coeffs[1],
        "k1G1_x": k1[0],
        "k1G1_y": k1[1],
        "k2G1_x": k2[0],
        "k2G1_y": k2[1],
        "one": pub_input[0],
        "out": pub_input[1],
        "gammaG2_x1": normalize(gamma_G2)[0].coeffs[0],
        "gammaG2_x2": normalize(gamma_G2)[0].coeffs[1],
        "gammaG2_y1": normalize(gamma_G2)[1].coeffs[0],
        "gammaG2_y2": normalize(gamma_G2)[1].coeffs[1],
        "deltaG2_x1": normalize(delta_G2)[0].coeffs[0],
        "deltaG2_x2": normalize(delta_G2)[0].coeffs[1],
        "deltaG2_y1": normalize(delta_G2)[1].coeffs[0],
        "deltaG2_y2": normalize(delta_G2)[1].coeffs[1],
    }
    output = template.substitute(variables)

with open("VerifierPublicInputGammaDelta.sol", "w") as f:
    f.write(output)

## r and s

In [24]:
print("Additional Trusted setup:")
print("-"*10)
delta_G1 = multiply(G1, int(delta))
gamma_G1 = multiply(G1, int(gamma))

print(f"[γ]G1 = {normalize(gamma_G1)}")
print(f"[δ]G1 = {normalize(delta_G1)}")


Additional Trusted setup:
----------
[γ]G1 = (3010198690406615200373504922352659861758983907867017329644089018310584441462, 4027184618003122424972590350825261965929648733675738730716654005365300998076)
[δ]G1 = (10744596414106452074759370245733544594153395043370666422502510773307029471145, 848677436511517736191562425154572367705380862894644942948681172815252343932)


In [25]:
print("Prover picks random r, s")
r = FP(12)
s = FP(13)

print(f"r = {r}")
print(f"s = {s}")

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

# U, V, W - witness already computed

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v == _w + ht

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

a = u + alpha + r * delta
b = v + beta + s * delta

c = ((beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1) + s * a + r * b - r * s * delta
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

assert a * b == alpha * beta + k * gamma + c * delta


Prover picks random r, s
r = 12
s = 13


In [26]:
r_delta_G1 = multiply(delta_G1, int(r))
s_delta_G1 = multiply(delta_G1, int(s))
s_delta_G2 = multiply(delta_G2, int(s))

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
A_G1 = add(A_G1, r_delta_G1)

B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)
B_G2 = add(B_G2, s_delta_G2)

B_G1 = evaluate_poly(V, tau_G1)
B_G1 = add(B_G1, beta_G1)
B_G1 = add(B_G1, s_delta_G1)

As_G1 = multiply(A_G1, int(s))
Br_G1 = multiply(B_G1, int(r))
rs_delta_G1 = multiply(delta_G1, int(-r*s))

C_G1 = add(Kw_delta_G1, HT_G1)
C_G1 = add(C_G1, As_G1)
C_G1 = add(C_G1, Br_G1)
C_G1 = add(C_G1, rs_delta_G1)

print(f"[A]G1 = {normalize(A_G1)}")
print(f"[B]G2 = {normalize(B_G2)}")
print(f"[C]G1 = {normalize(C_G1)}")

[A]G1 = (14614942786781395235323739941473784359708158727987721451462286880826505164913, 21619883166399935899647001848432357053098942563907854290524692662449417028110)
[B]G2 = ((319587749637165176148886793271196602371225601986791290743038843027662975010, 3799125525057959973194726334738911206921416163026998466480194981504341041059), (14804322218212369892528022736659815952157535301541362469519071425927767448738, 15320077395654528648820749046262517324306647091379407731658843025099270151841))
[C]G1 = (17701753435269740403921105970374551089935320911024962898677727136701795584514, 14346862312910190547529197118974415823038018600073367279111060787975692996852)


In [27]:
k1 = normalize(K_gamma_G1[0])
k2 = normalize(K_gamma_G1[1])


with open("VerifierPublicInputGammaDelta.sol.template", "r") as f:
    template = Template(f.read())
    variables = {
        "aG1_x": normalize(A_G1)[0],
        "aG1_y": normalize(A_G1)[1],
        "bG2_x1": normalize(B_G2)[0].coeffs[0],
        "bG2_x2": normalize(B_G2)[0].coeffs[1],
        "bG2_y1": normalize(B_G2)[1].coeffs[0],
        "bG2_y2": normalize(B_G2)[1].coeffs[1],
        "cG1_x": normalize(C_G1)[0],
        "cG1_y": normalize(C_G1)[1],
        "alphaG1_x": normalize(alpha_G1)[0],
        "alphaG1_y": normalize(alpha_G1)[1],
        "betaG2_x1": normalize(beta_G2)[0].coeffs[0],
        "betaG2_x2": normalize(beta_G2)[0].coeffs[1],
        "betaG2_y1": normalize(beta_G2)[1].coeffs[0],
        "betaG2_y2": normalize(beta_G2)[1].coeffs[1],
        "k1G1_x": k1[0],
        "k1G1_y": k1[1],
        "k2G1_x": k2[0],
        "k2G1_y": k2[1],
        "one": pub_input[0],
        "out": pub_input[1],
        "gammaG2_x1": normalize(gamma_G2)[0].coeffs[0],
        "gammaG2_x2": normalize(gamma_G2)[0].coeffs[1],
        "gammaG2_y1": normalize(gamma_G2)[1].coeffs[0],
        "gammaG2_y2": normalize(gamma_G2)[1].coeffs[1],
        "deltaG2_x1": normalize(delta_G2)[0].coeffs[0],
        "deltaG2_x2": normalize(delta_G2)[0].coeffs[1],
        "deltaG2_y1": normalize(delta_G2)[1].coeffs[0],
        "deltaG2_y2": normalize(delta_G2)[1].coeffs[1],
    }
    output = template.substitute(variables)

with open("VerifierPublicInputGammaDeltaRS.sol", "w") as f:
    f.write(output)